In [1]:
import pandas as pd
from playwright.async_api import async_playwright

In [2]:
BASE_URL = "https://www.hafen-hamburg.de/en/statistics"

PAGES = [
    'breakbulk',
    'bulk',
    'containerhandling',
    'seabornecargohandling'
]

DATA_DIR = '../data/hafen-hamburg-de/'

In [3]:
def to_dataframe(chart_data):
    [datasets, labels, names, _] = chart_data
    names = names[:len(datasets)]
    df = pd.DataFrame(datasets, index=pd.Index(names)).T.reset_index(drop=True)
    df['labels'] = labels

    return df

In [4]:
pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=True)
page = await browser.new_page()

In [5]:
for page_name in PAGES:
    print(f"Scraping {page_name}")
    url = f"{BASE_URL}/{page_name}"
    await page.goto(url)
    page_data = await page.evaluate('() => stats.slice(1)')

    for i, chart_data in enumerate(page_data):
        df = to_dataframe(chart_data)
        df.to_csv(f"{DATA_DIR}/{page_name}_{i}.csv", index=False)

Scraping breakbulk
Scraping bulk
Scraping containerhandling
Scraping seabornecargohandling


In [6]:
await browser.close()
pw.stop()

<coroutine object PlaywrightContextManager.__aexit__ at 0x7f5207604380>